# Data Import

In [14]:
import pandas as pd
billonarires_data = pd.read_csv("./data/Billionaires Statistics Dataset.csv")

In [15]:
billonarires_data.head(5)

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,...,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,67059887.0,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891


In [16]:
billonarires_data.columns

Index(['rank', 'finalWorth', 'category', 'personName', 'age', 'country',
       'city', 'source', 'industries', 'countryOfCitizenship', 'organization',
       'selfMade', 'status', 'gender', 'birthDate', 'lastName', 'firstName',
       'title', 'date', 'state', 'residenceStateRegion', 'birthYear',
       'birthMonth', 'birthDay', 'cpi_country', 'cpi_change_country',
       'gdp_country', 'gross_tertiary_education_enrollment',
       'gross_primary_education_enrollment_country', 'life_expectancy_country',
       'tax_revenue_country_country', 'total_tax_rate_country',
       'population_country', 'latitude_country', 'longitude_country'],
      dtype='object')

# Data Transformation

## Date Dimension

In [17]:
billonarires_data['date'] = pd.to_datetime(billonarires_data['date'])
dim_fecha = pd.DataFrame()
dim_fecha['year'] = billonarires_data['date'].dt.year
dim_fecha['month'] = billonarires_data['date'].dt.month
dim_fecha['day'] = billonarires_data['date'].dt.day
dim_fecha.head(1)

,year,month,day
0,2023,4,4


## Person Dimension

In [18]:
dim_persona = billonarires_data[['firstName', 'lastName', 'age', 'gender', 'title']]
dim_persona.head(1)

,firstName,lastName,age,gender,title
0,Bernard,Arnault,74.0,M,Chairman and CEO


## Location Dimension

In [28]:
dim_ubicacion = billonarires_data[['country', 'city']]
dim_ubicacion.head(1)

,country,city
0,France,Paris


## Organization Dimension

In [20]:
dim_organizacion = pd.DataFrame()
dim_organizacion['name'] = billonarires_data['source']
dim_organizacion['category'] = billonarires_data['industries']
dim_organizacion.head(1)

,name,category
0,LVMH,Fashion & Retail


## Fact Table

In [21]:
fact_table = billonarires_data[['finalWorth']]
fact_table.head(1)

,finalWorth
0,211000


# Database Connection

In [22]:
import pyodbc
from dotenv import load_dotenv
import os
load_dotenv()

SERVER = os.getenv('MS_SQL_SERVER_ADDRESS')
DATABASE = os.getenv('MS_SQL_DATABASE')
USERNAME = os.getenv('MS_SQL_USERNAME')
PASSWORD = os.getenv('MS_SQL_PASSWORD')

In [23]:
connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

In [24]:
conn = pyodbc.connect(connectionString)
cursor = conn.cursor()

# Tables Creation

## Date Dimension

In [29]:
dim_fecha.columns

Index(['year', 'month', 'day'], dtype='object')

In [26]:
cursor.execute("""
CREATE TABLE Dim_Date (
    dateId INT PRIMARY KEY,
    year INT,
    month INT,
    day INT
);
""")
cursor.commit()

## Location Dimension

In [30]:
dim_ubicacion.columns

Index(['country', 'city'], dtype='object')

In [31]:
cursor.execute("""
CREATE TABLE Dim_Location (
    locationId INT IDENTITY(1,1) PRIMARY KEY,
    country NVARCHAR(100) NOT NULL,
    city NVARCHAR(100) NOT NULL,
);
""")
cursor.commit()

## Person Dimension

In [32]:
dim_persona.columns

Index(['firstName', 'lastName', 'age', 'gender', 'title'], dtype='object')

In [33]:
cursor.execute("""
CREATE TABLE Dim_Person (
    personId INT IDENTITY(1,1) PRIMARY KEY,
    firstName NVARCHAR(100),
    lastName NVARCHAR(100),
    age INT,
    gender CHAR(1),
    title NVARCHAR(255),
);
""")
cursor.commit()

## Organization Dimension

In [34]:
dim_organizacion.columns

Index(['name', 'category'], dtype='object')

In [36]:
cursor.execute("""
CREATE TABLE Dim_Organization (
    organizationId INT IDENTITY(1,1) PRIMARY KEY,
    name NVARCHAR(255),
    category NVARCHAR(255)
);
""")
cursor.commit()

## Fact Table

In [39]:
cursor.execute("""
CREATE TABLE Worth_Fact (
    finalWorth DECIMAL(18,2) NOT NULL,
    
    -- Claves Foráneas
    dateId INT,
    personId INT,
    locationId INT,
    organizationId INT,
    
    -- Definición de claves foráneas
    CONSTRAINT FK_Fact_Date FOREIGN KEY (dateId) REFERENCES Dim_Date(dateId),
    CONSTRAINT FK_Fact_Person FOREIGN KEY (personId) REFERENCES Dim_Person(personId),
    CONSTRAINT FK_Fact_Location FOREIGN KEY (locationId) REFERENCES Dim_Location(locationId),
    CONSTRAINT FK_Fact_Organization FOREIGN KEY (organizationId) REFERENCES Dim_Organization(organizationId)
);
""")
cursor.commit()

## Diagram
![Diagrama de la BDD](imgs/diag-worth_fact.png)